<a href="https://colab.research.google.com/github/manuelboi/MLsec_project/blob/main/MLsec_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Models:
1. Fixing Data Augmentation to Improve Adversarial Robustness
2. Robust Learning Meets Generative Models: Can Proxy Distributions Improve Adversarial Robustness?
3. MMA Training: Direct Input Space Margin Maximization through Adversarial Training

In [ ]:
!pip install git+https://github.com/RobustBench/robustbench
'''
!pip install git+https://github.com/fra31/auto-attack
!pip install adversarial-robustness-toolbox[pytorch]
!pip install secml
!pip install torch
'''

from robustbench.utils import load_model
from robustbench.eval import benchmark
from robustbench.data import load_cifar10
from robustbench.model_zoo.enums import ThreatModel
#from autoattack import AutoAttack

x_test, y_test = load_cifar10(n_examples=5)


  Cloning https://github.com/RobustBench/robustbench to /tmp/pip-req-build-ito8u7kr
  Running command git clone --filter=blob:none --quiet https://github.com/RobustBench/robustbench /tmp/pip-req-build-ito8u7kr
  Resolved https://github.com/RobustBench/robustbench to commit 776bc95bb4167827fb102a32ac5aea62e46cfaab
  Preparing metadata (setup.py) ... done
  Using cached autoattack-0.1-py3-none-any.whl
Files already downloaded and verified


Matteo: Fixing Data Augmentation to Improve Adversarial Robustness

In [ ]:
# hide entire cell output
%%capture

try:
  import robustbench
  import smoothing
except:
  !pip install git+https://github.com/RobustBench/robustbench.git
  !git clone https://github.com/locuslab/smoothing.git
  # smoothing repository dependencies
  !conda create -n smoothing
  !conda activate smoothing
  !conda install pytorch torchvision cudatoolkit=10.0 -c pytorch
  !conda install scipy pandas statsmodels matplotlib seaborn
  !pip install setGPU

# scipy1.9.0 uses "binom_test()" instead of "binomtest()". Smoothing repository code uses "binom_test()"
# so a downgrade is needed
!pip install scipy==1.9.0


## Base model setup

In [ ]:
from robustbench.utils import load_model
from robustbench.data import load_cifar10
from smoothing.code.core import Smooth
import torch
import matplotlib.pyplot as plt


labels_dct = {0: "airplane", 1: "automobile", 2: "bird", 3: "cat", 4: "deer", 5: "dog", 6: "frog", 7: "horse", 8: "ship", 9: "truck"}
# use GPU if available, otherwise use CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

base_model = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='L2')
base_model.to(device)
x_test, y_test = load_cifar10(10)
x_test, y_test = x_test.to(device), y_test.to(device)
x, y = x_test[2], y_test[2]

if device != "cpu":
  x = x.cpu()
x_image = x.permute(1, 2, 0)
plt.imshow(x_image)
print(f"True class: {y.item()} ({labels_dct[y.item()]})")



## Smooth model setup
Training base model with Gaussian data augmentation: model learns on the perturbed images through Randomized Smoothing as well.

TODO: fix the script!

In [ ]:
try:
  import torchvision
except:
  !pip install torchvision

import smoothing
from torchvision.datasets import CIFAR10

import torchvision.transforms as transforms
import time
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import StepLR
import importlib.util
import re

# use of regular expression to fix ModuleNotFoundException for non-found libraries
prefix = " smoothing.code."
imports = ["datasets", "architectures", "archs", "train_utils"]
REGEX_REPLACEMENTS = [
    (r" datasets", prefix+imports[0]),
    (r" architectures", prefix+imports[1]),
    (r" archs", prefix+imports[2]),
    (r" train_utils", prefix+imports[3]),
]

def replace_file_text(file_path):
  with open(file_path, "r") as f:
    code = f.read()
  for old, new in REGEX_REPLACEMENTS:
    code = re.sub(old, new, code, count=1)
  with open(file_path, "w") as f:
    f.write(code)


# TODO: use only a function to replace text
# parse_args() in the train.py script make the whole script be executed.
# For our task, we need functions train() and test() only
replace_file_text("smoothing/code/train.py")
with open("smoothing/code/train.py", "r") as f:
  code = f.read()
code = code.replace("parser.parse_args()", "None")
with open("smoothing/code/train.py", "w") as f:
  f.write(code)

from smoothing.code.train import train, test

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

criterion = CrossEntropyLoss().cuda()
optimizer = SGD(base_model.parameters())
scheduler = StepLR(optimizer, step_size=30)

epochs = 5
sigma = 0.5     # gaussian noise standard deviation


for epoch in range(epochs):
  scheduler.step(epoch)
  before = time.time()
  train_loss, train_acc = train(trainloader, base_model, criterion, optimizer, epoch, sigma)
  test_loss, test_acc = test(testloader, base_model, criterion, sigma)
  after = time.time()


print(base_model.state_dict)


## Smooth model performances on perturbed images

In [ ]:
n_classes = 10

alpha = 0.05   # (1 - alpha) is the confidence level (in this case is 0.95)
n0 = 100        # number of samples for selection
n = 1000    # number of samples for estimation (certify) (too few samples but computation time is strongly affected with more)

smooth_model = Smooth(base_model, n_classes, sigma)

# loading test samples (10000 images)
x_test, y_test = load_cifar10()
x_test, y_test = x_test.cuda(), y_test.cuda()
print(x_test.shape)

top_classes = list()
for x, y in zip(x_test, y_test):
  top_class = smooth_model.predict(x, n0, alpha, batch_size=n0)
  top_class, radius = smooth_model.certify(x, n0, n, alpha, batch_size=n0)
  top_classes.append(top_class)

top_classes = torch.tensor(top_classes, dtype=torch.float64)
accuracy = torch.mean(top_classes == y, dtype=torch.float64)
print("Accuracy on smooth model: ", accuracy)

"""
top_class = smooth_model.predict(x, n0, alpha, batch_size=n0)
print(f"Top class predicted: {top_class} ({labels_dct[top_class]})")
print(f"True class of the perturbed samples: {y.item()} ({labels_dct[y.item()]})")

top_class, radius = smooth_model.certify(x, n0, n, alpha, batch_size=n0)
print(f"Top class: {top_class} ({labels_dct[top_class]}), certified radius: {radius}")
print(f"True class: {y.item()} ({labels_dct[y.item()]})")
"""


# NOTE: it looks like you need to use the "train" script to train the base
# classifier with gaussian data augmentation...



In [ ]:
# AutoAttack on model 1 with L2
model_1_L2 = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='L2')
benchmark(model_1_L2, threat_model=ThreatModel.L2, n_examples=5, eps=0.5)

Downloading...
From (original): https://drive.google.com/uc?id=1JX82BDVBNO-Ffa2J37EuB8C-aFCbz708
From (redirected): https://drive.google.com/uc?id=1JX82BDVBNO-Ffa2J37EuB8C-aFCbz708&confirm=t&uuid=279db62d-3494-40f7-a646-51a07c2997d4
To: /content/models/cifar10/L2/Rebuffi2021Fixing_70_16_cutmix_extra.pt
 41%|████      | 437M/1.07G [00:05<00:07, 89.0MB/s]

KeyboardInterrupt: 

In [ ]:
# AutoAttack on model 1 with Linf
model_1_Linf = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_extra', dataset='cifar10', threat_model='Linf')
benchmark(model_1_Linf, threat_model=ThreatModel.Linf, n_examples=5, eps=8/255)

Manuel: Robust Learning Meets Generative Models: Can Proxy Distributions Improve Adversarial Robustness?

In [ ]:
# AutoAttack on model 2 with L2
model_2_L2 = load_model(model_name='Sehwag2021Proxy', dataset='cifar10', threat_model='L2')
benchmark(model_2_L2, threat_model=ThreatModel.L2, n_examples=5, eps=0.5)

Files already downloaded and verified
Files already downloaded and verified
Clean accuracy: 100.00%
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.
initial accuracy: 100.00%
apgd-ce - 1/1 - 1 out of 5 successfully perturbed
robust accuracy after APGD-CE: 80.00% (total time 138.6 s)
apgd-t - 1/1 - 0 out of 4 successfully perturbed
robust accuracy after APGD-T: 80.00% (total time 1151.8 s)
fab-t - 1/1 - 0 out of 4 successfully perturbed
robust accuracy after FAB-T: 80.00% (total time 3219.1 s)
square - 1/1 - 0 out of 4 successfully perturbed
robust accuracy after SQUARE: 80.00% (total time 5953.8 s)
max L2 perturbation: 0.50000, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 80.00%
Adversarial accuracy: 80.00%
(1.0, 0.8)


In [ ]:
# AutoAttack on model 2 with Linf
model_2_Linf = load_model(model_name='Sehwag2021Proxy', dataset='cifar10', threat_model='Linf')
benchmark(model_2_Linf, threat_model=ThreatModel.Linf, n_examples=5, eps=8/255)

Downloading...
From (original): https://drive.google.com/uc?id=1QFA5fPMj2Qw4aYNG33PkFqiv_RTDWvzm
From (redirected): https://drive.google.com/uc?id=1QFA5fPMj2Qw4aYNG33PkFqiv_RTDWvzm&confirm=t&uuid=9337a4c9-b888-4b1e-995b-e5e7261acb8b
To: /content/models/cifar10/Linf/Sehwag2021Proxy.pt
100%|██████████| 1.11G/1.11G [00:13<00:00, 85.2MB/s]


Files already downloaded and verified
Clean accuracy: 100.00%
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/autoattack/checks.py", line 100, in check_dynamic
    sys.settrace(tracefunc)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/autoattack/checks.py", line 102, in check_dynamic
    sys.settrace(None)



initial accuracy: 100.00%
apgd-ce - 1/1 - 2 out of 5 successfully perturbed
robust accuracy after APGD-CE: 60.00% (total time 228.9 s)
apgd-t - 1/1 - 0 out of 3 successfully perturbed
robust accuracy after APGD-T: 60.00% (total time 1486.0 s)
fab-t - 1/1 - 0 out of 3 successfully perturbed
robust accuracy after FAB-T: 60.00% (total time 4150.4 s)
square - 1/1 - 0 out of 3 successfully perturbed
robust accuracy after SQUARE: 60.00% (total time 7608.7 s)
max Linf perturbation: 0.03137, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 60.00%
Adversarial accuracy: 60.00%


(1.0, 0.6)

Elia: MMA Training: Direct Input Space Margin Maximization through Adversarial Training

In [ ]:
# Autoattack on model 3 with L2
model_3_L2 = load_model(model_name='Ding2020MMA', dataset='cifar10', threat_model='L2')
benchmark(model_3_L2, threat_model=ThreatModel.L2, n_examples=5, eps=0.5)

In [ ]:
# AutoAttack on model 3 with Linf
model_3_Linf = load_model(model_name='Sehwag2021Proxy', dataset='cifar10', threat_model='Linf')
benchmark(model_3_Linf, threat_model=ThreatModel.Linf, n_examples=5, eps=8/255)